In [1]:
REPLAY_DELAY_MS = 200

In [2]:
import time
import argparse
import IrisBackendv3 as IB3
import IrisBackendv3.ipc as ipc
from IrisBackendv3.transceiver.logs import logger_setConsoleLevel as xcvrLoggerLevel
from IrisBackendv3.ipc.messages import (
    DownlinkedPacketsMessage, DownlinkedPacketsContent,
    DownlinkedPayloadsMessage, DownlinkedPayloadsContent
)
from IrisBackendv3.codec.payload import (
    TelemetryPayload, EventPayload, DownlinkedPayload, UplinkedPayload
)

import os  # noqa
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"  # noqa
from IrisBackendv3.data_standards import DataStandards  # noqa

from IrisBackendv3.codec.packet import parse_packet

from IrisBackendv3.data_standards.logs import (
    logger as DsLogger,
    logger_setConsoleLevel as DsLoggerLevel
)

from IrisBackendv3.logs import VALID_LOG_LEVELS

from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.transceiver.yamcs_helper import *

import warnings
from yamcs.client import YamcsClient, MDBClient, ArchiveClient  # type: ignore
from yamcs.tmtc.client import ProcessorClient  # type: ignore
from yamcs.tmtc.model import ParameterValue, CommandHistory  # type: ignore
from yamcs.core.auth import Credentials as YamcsCredentials  # type: ignore

import traceback
from typing import Any, Final, Tuple, List, Dict, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta
import os.path

import argparse

from collections import OrderedDict

import ulid
import numpy as np
import pandas as pd
import scapy.all as scp  # type: ignore

import time

import pickle


In [3]:
IB3.init_from_latest()

app = ipc.IpcAppHelper("YamcsReplaySandbox")
app.setLogLevel('VERBOSE')

2024-10-06 14:33:24: INFO     Successfully Loaded 29 Modules with 77 Commands, 262 Channels, and 92 Events from ./out/DataStandardsCache/IBv3_datastandards_01J9HGJHYVKR0BJV79JA1Y1BV1.dsc .
2024-10-06 14:33:24: NOTICE   IRIS Lunar Rover - YamcsReplaySandbox - IPC App Started


]0;YamcsReplaySandbox


In [4]:
from IrisBackendv3.storage.telemetry import DataSet
yamcs_dataset = DataSet.load_from_parquet('./out/fm1_data/yamcs_export/fm1_mission_archive.yamcs.parquet')
yamcs_dataset.data

2024-10-06 14:33:25: INFO     Successfully Loaded 29 Modules with 77 Commands, 262 Channels, and 92 Events from ./out/DataStandardsCache/IBv3_datastandards_01J9HGJHYVKR0BJV79JA1Y1BV1.dsc .


,*Command_Time1_queued,*Command_Time2_approved,*Command_Time3_radiated,*Command_Time4_onLander,*Command_alias,*Command_command,*Command_command_type,*Command_data,*Command_data_size,*Command_pathway,...,/Peregrine/MO/missionPhase,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_EnabledFet,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_avgCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_maxCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_EnabledFet,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_avgCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_maxCurrent,/Peregrine/PL1/SLIPsvc@3Status,/Peregrine/PL1/WIFIsvc@3Status,/Peregrine/payloads/iris/iris-payload-tm-rs422
time (UTC),,,,,,,,,,,,,,,,,,,,,
2024-01-08 08:11:09.670000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,Cruise,None,NaN,NaN,None,NaN,NaN,None,None,None
2024-01-08 08:16:22.251000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
2024-01-08 08:16:32.251000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
2024-01-08 08:16:42.251000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
2024-01-08 08:16:52.251000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-18 22:54:36.715000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
2024-01-18 22:54:46.715000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,INACTIVE,INACTIVE,None
2024-01-18 22:54:49.862000+00:00,NaT,NaT,NaT,NaT,None,None,None,None,NaN,None,...,None,None,NaN,NaN,None,NaN,NaN,None,None,None


In [5]:
# yamcs_dataset.save_to_excel('./out/fm1_data/yamcs_export/fm1_mission_archive.yamcs.xlsx')

In [6]:
from IrisBackendv3.storage.telemetry import DataSet
telem = DataSet.load_from_parquet('./out/fm1_data/yamcs_export/fm1_mission_archive.telem.parquet')
telem.data.shape

(232685, 267)

In [7]:
telem.save_to_excel('./out/fm1_data/yamcs_export/fm1_mission_archive.telem.xlsx', generate_telem_tables=True)

2024-10-06 14:33:33: DEBUG    Creating copy of data for Excel ...
2024-10-06 14:33:34: DEBUG    Building & saving Excel sheets . . .


Output()

2024-10-06 14:43:27: DEBUG    Formatting all Excel Sheets . . .

In [18]:
from IrisBackendv3.storage.dataset import _isolate_event_columns, _isolate_non_numeric_telem_columns
df = telem.data
cols = [*df.columns]
limit = 200
fwd_cols = list(set([
    *_isolate_event_columns(cols),
    *_isolate_non_numeric_telem_columns(cols),
    *[c for c in cols if not pd.api.types.is_numeric_dtype(df[c].dtype)]
]))
df[fwd_cols] = df[fwd_cols].ffill(limit=limit)

In [8]:
PARAM_HISTORY_PATH: Final[str] = "./out/fm1_data/yamcs_export/parameter_histories.pickle"
CMD_HISTORY_PATH: Final[str] = "./out/fm1_data/yamcs_export/command_history.pickle"

In [9]:
param_reload: Dict[str, List[ParameterValue]]
cmd_reload: List[CommandHistory]

In [10]:
# Fetch the raw data export:
with open(PARAM_HISTORY_PATH, 'rb') as f:
    param_reload = pickle.load(f)

In [11]:
with open(CMD_HISTORY_PATH, 'rb') as f:
    cmd_reload = pickle.load(f)

In [5]:
from IrisBackendv3.storage.commands import yamcs_history_to_commands, generate_command_row, extract_data_from_yamcs
commands = yamcs_history_to_commands(cmd_reload)
commands.data.shape

2024-10-05 20:43:42: INFO     Successfully Loaded 29 Modules with 77 Commands, 259 Channels, and 92 Events from ./out/DataStandardsCache/IBv3_datastandards_01J9F9PEE7GTARW8ERS29GZFPK.dsc .
2024-10-05 20:43:43: DEBUG    Reconstructing command data from 226 YAMCS entries . . . 


Output()

2024-10-05 20:43:43: DEBUG    Formatting data of 226 extracted commands . . . 


Output()

2024-10-05 20:43:43: DEBUG    Joining command rows . . . 
2024-10-05 20:43:43: DEBUG    Building dataset from (226, 15) df . . . 


(226, 15)

In [19]:
# Merge commands table into YAMCS table:
# Prefix all columns:
commands_prefixed = commands.data.rename(columns={
    c: '*Command_'+c
    for c in commands.data.columns
})
yamcs_dataset.data.join(commands.data, how='outer')


In [21]:
yd2 = yamcs_dataset.data.join(commands_prefixed, how='outer')
yd2[~yd2['*Command_command'].isna()]

,/Peregrine/FCPU/TMGR/TCS_DECKD_2/TCS_DECKD_2,/Peregrine/MO/missionPhase,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_EnabledFet,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_avgCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_maxCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_EnabledFet,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_avgCurrent,/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_maxCurrent,/Peregrine/PL1/SLIPsvc@3Status,/Peregrine/PL1/WIFIsvc@3Status,...,*Command_command,*Command_command_type,*Command_data,*Command_data_size,*Command_pathway,*Command_source,*Command_yamcs_Name,*Command_yamcs_Queue,*Command_yamcs_User,*Command_yamcs_comment
time,,,,,,,,,,,,,,,,,,,,,
2024-01-09 09:03:00.653000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ReportStatus: WatchDogInterface_RequestStatusR...,WatchdogCommandPayload,0xf61057,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Req...,iris_independent,iris,Requesting Iris send a single 48 Byte health &...
2024-01-09 13:30:00.498000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,setup: WatchDogInterface_SwitchToServiceMode(0...,WatchdogCommandPayload,0xec1077,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Swi...,iris_dependent,iris,None
2024-01-09 13:36:03.703000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,power-on: WatchDogInterface_PrepareForDeployme...,WatchdogCommandPayload,0x011060,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Pre...,iris_hazardous,iris,None
2024-01-09 13:38:23.818000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,batt-en-off: WatchDogInterface_SetBatteryConne...,WatchdogCommandPayload,0xf91000,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Set...,iris_dependent,iris,None
2024-01-09 13:39:36.084000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WatchDogInterface_SetBatteryLatch(0x10FB)[latc...,WatchdogCommandPayload,0xfb10aa,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Set...,iris_dependent,iris,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15 05:23:05.847000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,setup: WatchDogInterface_SwitchToServiceMode(0...,WatchdogCommandPayload,0xec1077,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Swi...,iris_dependent,iris,None
2024-01-15 05:23:51.236000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,setup: WatchDogInterface_SwitchToServiceMode(0...,WatchdogCommandPayload,0xec1077,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Swi...,iris_dependent,iris,None
2024-01-15 05:24:56.158000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,transit: WatchDogInterface_SwitchToKeepAliveMo...,WatchdogCommandPayload,0xeb1077,3.0,DataPathway.WIRED,DataSource.NONE,/Peregrine/payloads/iris/WatchDogInterface_Swi...,iris_independent,iris,None


In [22]:
param = param_reload['/Peregrine/payloads/iris/iris-payload-tm-rs422'][0]
from IrisBackendv3.transceiver.yamcs_helper import iris_telem_param_to_packet
packet = iris_telem_param_to_packet(param, True)
time = param.generation_time
from IrisBackendv3.storage.telemetry import packet_to_telem_rows
packet_to_telem_rows(time, packet)

,*Packet_type,*Packet_str,*Packet_valid,*Packet_source,*Packet_pathway,*Packet_size,*Packet_bytes,WatchdogDetailedStatus_Io_ChargingStatus1,WatchdogDetailedStatus_Io_ChargingStatus2,WatchdogDetailedStatus_Io_ChargerState,...,MetaModWatchdogMonitor_FpgaNRst,MetaModWatchdogMonitor_RadioNRst,MetaModWatchdogMonitor_RadioOn,MetaModWatchdogMonitor_BmsBoot,MetaModWatchdogMonitor_LatchSet,MetaModWatchdogMonitor_LatchReset,MetaModWatchdogMonitor_BattStat,MetaModRoverPower_LanderVoltage_FusedEst,MetaModRoverPower_FullSystemSwitch_FaultState,MetaModTransitPower_VHeaterLive
time (UTC),,,,,,,,,,,,,,,,,,,,,
2024-01-09 08:53:40.027000+00:00,WatchdogDetailedStatusPacket,#2 [RS_KEEP_ALIVE + NOT_DEPLOYED] \tCHARGER: O...,True,YAMCS,WIRED,47.0,0xd500081800020200003800000000006600428086c000...,0.0,0.0,OFF,...,0.0,0.0,0.0,0.0,2.0,2.0,2.0,29.611951,OFF/FAULT,29.517881


In [23]:
params = param_reload['/Peregrine/payloads/iris/iris-payload-tm-rs422'][:4000]
from IrisBackendv3.transceiver.yamcs_helper import iris_telem_param_to_packet
packets = [iris_telem_param_to_packet(p, True) for p in params]
times = [p.generation_time for p in params]
rows = [packet_to_telem_rows(time, packet) for time, packet in zip(times, packets)]
rows[-5:]

[                                                 *Packet_type  \
 time (UTC)                                                      
 2024-01-09 13:51:26.361000+00:00  WatchdogTvacHeartbeatPacket   
 
                                                                         *Packet_str  \
 time (UTC)                                                                            
 2024-01-09 13:51:26.361000+00:00  SERVICE:\t[Heat: ON, Ctrl: ON] \t283.4°K -> 27...   
 
                                   *Packet_valid *Packet_source  \
 time (UTC)                                                       
 2024-01-09 13:51:26.361000+00:00           True          YAMCS   
 
                                  *Packet_pathway  *Packet_size  \
 time (UTC)                                                       
 2024-01-09 13:51:26.361000+00:00           WIRED          24.0   
 
                                                                       *Packet_bytes  \
 time (UTC)                             

In [160]:
from IrisBackendv3.codec.payload import CommandPayload, settings

In [16]:
set([c.split('_')[0] for c in telem.columns if '_' in c])

{'ActiveRateGroup-RateGroupHiFreq',
 'ActiveRateGroup-RateGroupLowFreq',
 'ActiveRateGroup-RateGroupMedFreq',
 'CommandDispatcher',
 'GdsPackets',
 'GroundInterface',
 'Imu',
 'MetaModImu',
 'MetaModLatency',
 'MetaModPeregrine',
 'MetaModRoverPower',
 'MetaModTemps',
 'MetaModTransitPower',
 'MetaModWatchdogMonitor',
 'MotorControl',
 'N',
 'NetworkManager',
 'Peregrine',
 'WatchDogInterface',
 'WatchdogCommandResponse',
 'WatchdogDetailedStatus',
 'WatchdogHeartbeat',
 'WatchdogHeartbeatTvac'}

In [7]:
# Group all entries by generation time & sort (so they replay correctly):
app.logger.notice(
    f"Recovered {len([*param_reload.keys()])} YAMCS parameters with {(n_pts:=sum(len(vs) for vs in param_reload.values()))} total values."
)

# Clump by common generation_time:
time_dict: Dict[datetime, List[ParameterValue]] = dict()
for name, entries in param_reload.items():
    for entry in entries:
        if entry.generation_time not in time_dict:
            time_dict[entry.generation_time] = []
        time_dict[entry.generation_time].append(entry)

# Sort by generation time:
time_dict = dict(sorted(time_dict.items()))

def log_time_dict_stats(time_dict: Dict[datetime, List[ParameterValue]]):
    app.logger.info(
        f"Collected {sum(len(entries) for entries in time_dict.values())}"
        f"... clumped into {len(time_dict)} distinct time points from "
        f"{[*time_dict.keys()][0]:%Y-%m-%d %H:%M:%S%z} to "
        f"{[*time_dict.keys()][-1]:%Y-%m-%d %H:%M:%S%z}."
    )

    # Check for wrong "2000" bug with YAMCS timestamps on PM1
    # (no timestamp given and just assigned to some time close to 2000):
    app.logger.warning(
        f"Of the {(nt:=len(time_dict))} time points recovered, "
        f"{(ntb:=len([k for k in time_dict.keys() if k.year < 2020]))} ({100*ntb/nt:.3f}%) "
        f"are from before 2020 and are likely victim to the '2000' bug. "
        f"These {(n_bad:=sum(len(vs) for k, vs in time_dict.items() if k.year < 2024))} "
        f"datapoints of the {n_pts} total datapoints ({100*n_bad/n_pts:.3f}%) will be ignored. "
        f"The parameters affected are: {set(v.name for k,vs in time_dict.items() if k.year < 2020 for v in vs)}"
    )
log_time_dict_stats(time_dict)

# For some parameters duplicate identical entries (same time, same value) were
# fetched from YAMCS. Find and remove these:
n_entries_raw = sum(len(entries) for entries in time_dict.values())
for time in time_dict.keys():
    # Can't directly do ParameterValue comparison, but can compare their
    # `__dict__` s which contain all the core data we care about:
    unique_dicts: List[Dict] = []
    unique_entries: List[ParameterValue] = []
    for entry in time_dict[time]:
        if entry.__dict__ not in unique_dicts:
            unique_dicts.append(entry.__dict__)
            unique_entries.append(entry)
    time_dict[time] = unique_entries
n_entries_dedup = sum(len(entries) for entries in time_dict.values())
n_removed = n_entries_raw - n_entries_dedup
app.logger.notice(
    f"Duplicate Removal Complete. "
    f"Removed {n_removed} ({n_removed/n_entries_raw*100:.1f}%, <=70-80% expected). "
    f"Num. entries {n_entries_raw} -> {n_entries_dedup}. After removal:"
)
log_time_dict_stats(time_dict)

2024-10-02 11:28:22: NOTICE   Recovered 13 YAMCS parameters with 1217411 total values.
2024-10-02 11:28:42: INFO     Collected 1217411... clumped into 150719 distinct time points from 2001-05-09 22:19:12-0400 to 2024-03-04 09:18:01-0500.
2024-10-02 11:28:42: WARNING  Of the 150719 time points recovered, 412 (0.273%) are from before 2020 and are likely victim to the '2000' bug. These 734 datapoints of the 1217411 total datapoints (0.060%) will be ignored. The parameters affected are: {'/Peregrine/FCPU/TMGR/TCS_DECKD_2/TCS_DECKD_2', '/Peregrine/PL1/SLIPsvc@3Status', '/Peregrine/PL1/WIFIsvc@3Status'}
2024-10-02 11:29:09: NOTICE   Duplicate Removal Complete. Removed 882036 (72.5%, <=70-80% expected). Num. entries 1217411 -> 335375. After removal:
2024-10-02 11:29:09: INFO     Collected 335375... clumped into 150719 distinct time points from 2001-05-09 22:19:12-0400 to 2024-03-04 09:18:01-0500.
2024-10-02 11:29:09: WARNING  Of the 150719 time points recovered, 412 (0.273%) are from before 2

In [8]:
# Builder functions:
def recovered_data_to_packets(data: List[ParameterValue]) -> List[Packet] | None:
    packets: List[Packet] | None = None
    # Build packets:
    # Extract and process any Iris parameters into telem:
    packets = [
        iris_telem_param_to_packet(p) for p in data
        if p.name in IRIS_TELEM_PARAMS
    ]

    # Lump all the rest into a packet of Peregrine telem:
    peregrine_params = [
        p for p in data if p.name not in IRIS_TELEM_PARAMS
    ]
    if len(peregrine_params) > 0:
        packets.append(peregrine_telem_params_to_packet(peregrine_params))
    
    # Make PMCC_RX align with generation_time (the past when this happened) and
    # not the present so the rest of the stack thinks this is all happening in
    # the past:
    # TODO: Actually recover the real PMCC_RX. We necessarily saved this data
    # in IPC logs but it will be hairy to extract and correlate to this data. For now, assume reception=generation.
    for packet in packets:
        for payload in packet.payloads.all_payloads:
            if isinstance(payload, DownlinkedPayload):
                if payload.downlink_times.amcc_rx is not None:
                    # reception_time (seems to not be provided):
                    payload.downlink_times.amcc_rx = payload.downlink_times.amcc_rx
                else:
                    # generation_time:
                    payload.downlink_times.amcc_rx = payload.downlink_times.lander_rx

    return packets

In [9]:
# Convert to the H5 Database format that we've used for previous replay archives:
import scripts.testing._peregrine_tvac_fetcher as ptf
from IrisBackendv3.storage.dataset import DataSet

# AMCC doesn't provide generation time, so this really is all the meaningful
# data we get through YAMCS:
yamcs_data_dict: Dict[str, List[Tuple[datetime, Any]]] = dict()
for params in time_dict.values():
    for p in params:
        # Ignore params not in list (MO/timeline) since we can't handle those anyway:
        if p.name not in ALL_YAMCS_PARAMS:
            continue
        if p.name not in yamcs_data_dict:
            yamcs_data_dict[p.name] = []
        yamcs_data_dict[p.name].append((p.generation_time, p.eng_value))

# Return as DataSet:
dataset = DataSet.from_tuples_dict(yamcs_data_dict, time_tolerance=pd.Timedelta('1ms'))

In [10]:
from IrisBackendv3.storage.telemetry import peregrine_data_to_telem
yamcs_dataset = dataset
# peregrine_data_to_telem(yamcs_dataset)

In [11]:
from time import time

In [12]:
from IrisBackendv3.storage.telemetry import extract_iris_packets,generate_peregrine_packets
rx_is_generation_time = True
iris_times, iris_packets = extract_iris_packets(
    yamcs_dataset, rx_is_generation_time
)
peregrine_times, peregrine_packets = generate_peregrine_packets(
    yamcs_dataset, rx_is_generation_time
)
times = [*iris_times, *peregrine_times]
packets = [*iris_packets, *peregrine_packets]

2024-10-02 11:29:17: WARNING  Had to abort packet parsing due to the following exception: `Traceback (most recent call last):
  File "/Users/connorcolombo/Documents/PKBACK/GitHub/CubeRoverPackage/Apps/GroundSoftware/IrisBackendv3/codec/packet_classes/iris_common.py", line 366, in decode
    payloads = extract_downlinked_payloads(
  File "/Users/connorcolombo/Documents/PKBACK/GitHub/CubeRoverPackage/Apps/GroundSoftware/IrisBackendv3/codec/payload_collection.py", line 875, in extract_downlinked_payloads
    payload, VLP = payload_type.process(VLP, endianness_code)
  File "/Users/connorcolombo/Documents/PKBACK/GitHub/CubeRoverPackage/Apps/GroundSoftware/IrisBackendv3/codec/payload.py", line 125, in process
    payload = cls.decode(
  File "/Users/connorcolombo/Documents/PKBACK/GitHub/CubeRoverPackage/Apps/GroundSoftware/IrisBackendv3/codec/payload.py", line 742, in decode
    module = settings['STANDARDS'].modules[module_id]
  File "/Users/connorcolombo/Documents/PKBACK/GitHub/CubeRoverPa

In [ ]:
times, values = [*zip(*yamcs_data_dict['/Peregrine/FCPU/TMGR/TCS_DECKD_2/TCS_DECKD_2'])]
val_types = set(type(v) for v in values)
val_types.pop()

float

In [ ]:
name = '/Peregrine/FCPU/TMGR/TCS_DECKD_2/TCS_DECKD_2'
times, values = [*zip(*yamcs_data_dict[name])]
time_name = 'time'  # DataSet requires this to be the same across all


In [ ]:
df = dataset.data
df.size

3374992775

In [46]:
# Setup IPC:
manager = ipc.IpcAppManagerSync(socket_specs={
    'pub': ipc.SocketSpec(
        sock_type=ipc.SocketType.PUBLISHER,
        port=ipc.Port.TRANSCEIVER_PUB,
        topics=[ipc.Topic.DL_PACKETS],
        publish_only=True
    )
})

2024-09-28 21:39:23: VERBOSE  Using default binding scheme (`bind=False`) for `socket_type=<SocketType.PUBLISHER: 2>`.
2024-09-28 21:39:23: INFO     Created a `SocketType.PUBLISHER` connected to `tcp://127.0.0.1` on ports `[<Port.TRANSCEIVER_PUB: 61453>]` (`0xF00D`).


In [48]:
import asyncio
# Replay values into IPC:
# Shorthand aliases (for legacy compat.):
n_packets_sent: int = 0
for generation_time, param_vals in time_dict.items():
    if generation_time.year < 2020:
        # Corrupted datapoint from YAMCS. Skip:
        continue
    try:
        packets = recovered_data_to_packets(param_vals)
        if packets is None:
            continue
        # Emit packets:
        app.logger.notice(
            f"Got {len(param_vals)} params. "
            f"Forwarding {len(packets)} packets . . ."
        )
        msg = DownlinkedPacketsMessage(DownlinkedPacketsContent(
            packets=packets
        ))
        manager.send_to('pub', msg, subtopic_bytes=b'irisyamcs')
        n_packets_sent += len(packets)
        app.logger.notice(
            f"[{n_packets_sent:5d}] "
            f"Sent {msg.content.simple_str()} -> {ipc.Topic.DL_PACKETS}"
        )
    except Exception as e:
        app.logger.error(
            f"Uncaught exception in encoding packet(s): `{packets}`. \n"
            f"Exception: `{e}`."
        )
        trace = traceback.format_exc()
        logger.verbose(
            f"\t > The stack trace of this error was: `{trace}`."
        )
    await asyncio.sleep(REPLAY_DELAY_MS/1000)

2024-09-28 21:50:49: WARNING  param.name='/Peregrine/MO/timeline' is not a known Peregrine telem name in: LANDER_PARAMS_YAMCS_GDS_MAP={'/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_EnabledFet': 'IrisOperationalEnabledFet', '/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_EnabledFet': 'IrisReleaseEnabledFet', '/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_avgCurrent': 'IrisOperationalAvgCurrent', '/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_avgCurrent': 'IrisReleaseAvgCurrent', '/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Operational_maxCurrent': 'IrisOperationalMaxCurrent', '/Peregrine/PL1/LSS1_Derived/LSS1_HK_Derived/Iris_Release_maxCurrent': 'IrisReleaseMaxCurrent', '/Peregrine/PL1/SLIPsvc@3Status': 'SlipServiceStatus', '/Peregrine/PL1/WIFIsvc@3Status': 'WifiServiceStatus', '/Peregrine/FCPU/TMGR/TCS_DECKD_1/TCS_DECKD_1': 'DeckD1TempCelsius', '/Peregrine/FCPU/TMGR/TCS_DECKD_2/TCS_DECKD_2': 'DeckD2TempCelsius', '/Peregrine/FCPU/

CancelledError: 

In [61]:
# ! TODO: (WORKING-HERE): Port over into variant of h5_transceiver with the above loop serving as `_async_read_core`. That's pretty much it.
# ... or just make an H5 export and add PMCC_RX fix and commanding to H5

In [50]:
del app
del manager

NameError: name 'app' is not defined